In [ ]:
#installations
#!pip install --quiet SimpleITK
#Anaconda Powershell
#:L
#conda activate data_processing
#jupyter serverextension enable -- py jupyter_http_over_ws
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8892 --NotebookApp.port_retries=0

In [ ]:
#imports
import pandas as pd
import numpy as np
import os
import sklearn
import datetime
from datetime import datetime
from sklearn.model_selection import StratifiedKFold

#print(os.getcwd())

In [ ]:
#functions

#apply the labels to the model inputs dataset
def label_patient(row, df_refer):
  #get info on single patient
  patient = row['Patient']
  #get from annotations
  label = df_refer[df_refer['PatientID'] == patient]['Rajs impression'].iloc[0]
  #return
  return label

#encode into numerical annotations
def encode_patient(row, enc_dict):
  #return the label
  try:
    label = enc_dict[row['Annotation']]
  except:
    #if issue
    label = pd.NA
    print('Missing:', row['Patient'])
    #return
  return label

#seperate into independent folds
def get_folds(df, kfold, stratify):
  #init
  skf = StratifiedKFold(n_splits = kfold, shuffle = True, random_state = 42)
  #set the target
  target = df.loc[:, stratify].astype('category')
  #init
  df_ls = []
  #iterate
  for i, (_, fold_index) in enumerate(skf.split(df, target)):
    #set
    df_fold = df.loc[fold_index,:]
    #insert
    df_fold['KFold'] = i + 1
    #append
    df_ls.append(df_fold)
  #return
  return  pd.concat(df_ls).reset_index(drop = True)

In [ ]:
#params

#kfolds
kfolds = 5
#stratification column
stratify = 'Annotation_Label'
#the annotations path
ann_path = 'AnishSalvi/ImageRx/PAD-Net/ViT_classification/dataset/GroundTruthInterpretation--Raj&Luke.xlsx'
#path to the model inputs
data_path = 'AnishSalvi/ImageRx/PAD-Net/vRAD_Dataset_Inputs/Norm-256-256-256/vRAD_Dataset_Inputs.xlsx'
#specify the encoding dictionary for clinical annotations
ann_dict = {
    'Control / Non-PAD': 0,
    'Control/Non-PAD': 0,
    'PAD, <50% stenosis': 1,
    'PAD, >50% stenosis': 2,
    'X Unable to Interpret X': pd.NA
}
#specify the encoding dictionary for PAD annotations
pad_dict = {
    'Control / Non-PAD': 0,
    'Control/Non-PAD': 0,
    'PAD, <50% stenosis': 1,
    'PAD, >50% stenosis': 1,
    'X Unable to Interpret X': pd.NA
}
#specify the encoding dictionary for severity annotations
sev_dict = {
    'Control / Non-PAD': 0,
    'Control/Non-PAD': 0,
    'PAD, <50% stenosis': 0,
    'PAD, >50% stenosis': 1,
    'X Unable to Interpret X': pd.NA
}
save_folder = 'AnishSalvi/ImageRx/PAD-Net/ViT_classification/dataset/'

In [ ]:
#main script

#specify the annotations file (ignore first few rows)
df_refer = pd.read_excel(ann_path, skiprows = 2)
#specify the path to the model inputs
df = pd.read_excel(data_path)
#need to append the annotations to df
df['Annotation'] = df.apply(label_patient, axis = 1, args = (df_refer, ))
#print
print(df['Annotation'].value_counts(dropna = False))
#convert the annotations into model labels
df['Annotation_Label'] = df.apply(encode_patient, axis = 1, args = (ann_dict, ))
#can now encode the labels for PAD
df['PAD_Label'] = df.apply(encode_patient, axis = 1, args = (pad_dict, ))
#now encode for severity
df['Severity_Label'] = df.apply(encode_patient, axis = 1, args = (sev_dict, ))
#now remove the annotations without a label
df = df[df['Annotation_Label'].notna()].reset_index(drop = True)
#now get each independent fold // can also create an independent test set later!!!!
df = get_folds(df, kfolds, stratify)
#now save the results somewhere
df.to_pickle(save_folder + 'dataset' + datetime.now().strftime("-%Y-%m-%d-%H-%M-%S") + '.pkl')

PAD, >50% stenosis         84
X Unable to Interpret X    47
Control/Non-PAD            36
PAD, <50% stenosis         21
Control / Non-PAD           5
NaN                         1
Name: Annotation, dtype: int64
Missing: 1331806000
Missing: 1331806000
Missing: 1331806000
